In [1]:
import snscrape.modules.twitter as sntwitter
import pandas as pd

In [2]:
import datetime
begin_date= datetime.datetime(2021, 6, 12, 0, 0, 0)
end_date = datetime.datetime(2022, 6, 29, 0, 0, 0)
one_day = datetime.timedelta(days=1)

In [17]:
get_query()

'#COVID19 OR #CoronaVirus OR #Pandemic since:2022-06-29 until:2022-06-30 filter:verified'

In [4]:
all_tweets = []

In [5]:
# 2020-01-01, 2022-06-29
def get_query():
    '''
    Query for tweets of one day only
    '''
    QUERY = "#COVID19 OR #CoronaVirus OR #Pandemic since:{} until:{} filter:verified"
    if end_date>=begin_date:
        return QUERY.format(begin_date.date(), (begin_date+one_day).date())
    else:
        return "-1"


def scrape_tweets(QUERY):
    '''
    Get 275 tweets per day
    '''
    count = 275
    tweets = []
    for tweet in sntwitter.TwitterSearchScraper(QUERY).get_items():
            if len(tweets) == count:
                break
            if(tweet.lang=='en'):
                media_shared = 0 if (tweet.media) is None else len(tweet.media)
                tweets.append([tweet.rawContent, tweet.date, tweet.user.location, tweet.likeCount, tweet.sourceLabel, 
                           media_shared, tweet.retweetCount])
    return tweets

In [20]:
while True:
    QUERY = get_query()
    if QUERY != "-1":
        temp = scrape_tweets(QUERY)
        for i in temp:
            all_tweets.append(i)
        begin_date += one_day
    else:
        break
    if end_date == begin_date:
        break

In [ ]:
tweets[0].content, tweets[99].date, tweets[0].user.location, tweets[0].likeCount, tweets[0].sourceLabel.split()[1:], len(tweets[4].media), tweets[0].retweetCount 
# no of media

In [21]:
len(all_tweets)

105325

In [23]:
(all_tweets[105320])

['The Italian Ambassador Antonino Maggiore was honored to join the PS of MoH Prof. Lackon Kasonka, at the reception of 500,000 doses of #Pfizer vaccines, donated by the Italian Government to the MoH of #Zambia, in a combine effort to defeat the #coronavirus\n🇮🇹🇿🇲\n\n#vaccinesharing https://t.co/oMzUvnmOp1',
 datetime.datetime(2022, 6, 29, 15, 38, 31, tzinfo=datetime.timezone.utc),
 'Lusaka, Zambia',
 20,
 'Twitter for Android',
 1,
 1]

In [24]:
df = pd.DataFrame(all_tweets, columns=['content', 'date', 'location', 'like_count', 'source', 'total_media_shared', 'retweet_count'])

In [25]:
df['date'].dt.date.value_counts()

2021-06-12    275
2022-02-19    275
2022-02-28    275
2022-02-27    275
2022-02-26    275
             ... 
2021-10-15    275
2021-10-14    275
2021-10-13    275
2021-10-12    275
2022-06-29    275
Name: date, Length: 383, dtype: int64

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105325 entries, 0 to 105324
Data columns (total 7 columns):
 #   Column              Non-Null Count   Dtype              
---  ------              --------------   -----              
 0   content             105325 non-null  object             
 1   date                105325 non-null  datetime64[ns, UTC]
 2   location            105325 non-null  object             
 3   like_count          105325 non-null  int64              
 4   source              105325 non-null  object             
 5   total_media_shared  105325 non-null  int64              
 6   retweet_count       105325 non-null  int64              
dtypes: datetime64[ns, UTC](1), int64(3), object(3)
memory usage: 5.6+ MB


In [ ]:
import geopy
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent = "http")
loc = geolocator.geocode('NY/Delhi/Rio/Beijing/Nairobi/Joburg/London')
print(loc.address.split(",")[-1])
# u'Chicago, Cook County, Illinois, United States of America'
loc = geolocator.geocode('San Francisco California')
print(loc.address) 
# u'SF, California, United States of America'

In [27]:
df.to_csv("dataframe_2.csv", index=False)

In [13]:
df1 = pd.read_csv('dataframe_1.csv')
df2 = pd.read_csv('dataframe_2.csv')

In [14]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140278 entries, 0 to 140277
Data columns (total 7 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   content             140278 non-null  object
 1   date                140278 non-null  object
 2   location            130554 non-null  object
 3   like_count          140278 non-null  int64 
 4   source              140278 non-null  object
 5   total_media_shared  140278 non-null  int64 
 6   retweet_count       140278 non-null  int64 
dtypes: int64(3), object(4)
memory usage: 7.5+ MB


In [15]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105325 entries, 0 to 105324
Data columns (total 7 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   content             105325 non-null  object
 1   date                105325 non-null  object
 2   location            98725 non-null   object
 3   like_count          105325 non-null  int64 
 4   source              105325 non-null  object
 5   total_media_shared  105325 non-null  int64 
 6   retweet_count       105325 non-null  int64 
dtypes: int64(3), object(4)
memory usage: 5.6+ MB


In [16]:
df_concat = pd.concat([df1, df2], axis=0)
df_concat = df_concat.reset_index(drop=True)

In [17]:
df_concat.drop_duplicates()

,content,date,location,like_count,source,total_media_shared,retweet_count
0,1/ *NEW PAPERS ALERT*\n\nVery proud to share t...,2020-01-01 19:51:05+00:00,"Stanford, CA",15,Twitter Web App,0,4
1,"Drawing from actual events, #Event201 identifi...",2020-01-02 15:00:00+00:00,Baltimore,1,TweetDeck,0,0
2,Interesting article @PNAS on whether the Justi...,2020-01-05 22:46:12+00:00,Kumeyaay unceded land.,3,Twitter Web Client,0,1
3,@arambaut @WHO But batteries of tests have bee...,2020-01-05 19:05:20+00:00,"New York, NY",1,Twitter Web App,0,0
4,#Pandemic players: what do you do if you draw ...,2020-01-06 15:32:57+00:00,Kingdom of Fife,3,Twitter for Android,0,0
...,...,...,...,...,...,...,...
245598,The Italian Ambassador Antonino Maggiore was h...,2022-06-29 15:38:31+00:00,"Lusaka, Zambia",20,Twitter for Android,1,1
245599,PAHO is recommending a fourth dose of the vacc...,2022-06-29 15:38:24+00:00,Port-au-Prince/Miami,4,Twitter Web App,0,2
245600,#elmo breaks down the #covid19 vaccine for kid...,2022-06-29 15:36:26+00:00,"Long Island, NY",0,SocialNewsDesk,1,0
245601,"#Greece confirmed 16,115 new #coronavirus infe...",2022-06-29 15:35:46+00:00,"Athens, Greece",0,Twitter Web App,1,0


In [19]:
df_concat.drop_duplicates(inplace=True)

In [22]:
df_concat.drop_duplicates(subset=['content'], inplace=True)

In [28]:
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 243185 entries, 0 to 245602
Data columns (total 7 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   content             243185 non-null  object
 1   date                243185 non-null  object
 2   location            226918 non-null  object
 3   like_count          243185 non-null  int64 
 4   source              243185 non-null  object
 5   total_media_shared  243185 non-null  int64 
 6   retweet_count       243185 non-null  int64 
dtypes: int64(3), object(4)
memory usage: 14.8+ MB


In [29]:
df_concat.to_csv('245k_tweets', index = False)